## Using our SQL skills to answer business questions
We will be using the **[Chinook database](https://www.sqlitetutorial.net/sqlite-sample-database/)** which is provided as a SQLite database file called `chinook.db` to answer some business questions. The database schema  in PDF can be downloaded [SQLite Sample database schema(diagram)](https://www.sqlitetutorial.net/wp-content/uploads/2018/03/sqlite-sample-database-diagram.pdf). The color version is found at [SQLite Sample database schema(color)](https://www.sqlitetutorial.net/wp-content/uploads/2018/03/sqlite-sample-database-diagram-color.pdf).

### Connect Jupyter Notebook to the database file

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

### Let's query the database and see all tables and views in `chinook.db`
Write a query to return information on the tables and views in the database

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ('table', 'view');

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


Write one or two queries to get familiar with the tables and to practice running SQL in this interface. Use the schema diagram for reference.

### 1 - Each Country's Best Customer

Create a query to find the **customer from each country that has spent the most money at our store**, ordered alphabetically by country. Your query should return the following columns, in order:

* `country` - The name of each country that we have a customer from.
* `customer_name` - The first_name and last_name of the customer from that country with the most total purchases, separated by a space, eg `Luke Skywalker`.
* `total_purchased` - The total dollar amount that customer has purchased.

In [3]:
%%sql
WITH 
    all_customers AS (SELECT * FROM customer),

    purchases AS (SELECT 
                 SUM(il.quantity * il.unit_price) total_purchases, 
                 c.customer_id
                 FROM invoice_line il 
                 INNER JOIN invoice i ON il.invoice_id = i.invoice_id
                 INNER JOIN customer c ON c.customer_id = i.customer_id
                 GROUP BY i.customer_id)
                 
SELECT 
    all_cus.country country,
    all_cus.first_name || ' ' || all_cus.last_name AS customer_name,
    ROUND(MAX(p.total_purchases), 2) total_purchased 
    FROM all_customers all_cus
    INNER JOIN purchases p ON p.customer_id = all_cus.customer_id
    GROUP BY country 
    ORDER BY country
    LIMIT 5;

Done.


country,customer_name,total_purchased
Argentina,Diego Gutiérrez,39.6
Australia,Mark Taylor,81.18
Austria,Astrid Gruber,69.3
Belgium,Daan Peeters,60.39
Brazil,Luís Gonçalves,108.9


### 2 - Combining Multiple Joins with Subqueries

Write a query that returns the **top 5 albums**, as calculated by the **number of times a track from that album has been purchased**. Your query should be sorted from most tracks purchased to least tracks purchased and return the following columns, in order:
* `album`, the title of the album
* `artist`, the artist who produced the album
* `tracks_purchased` the total number of tracks purchased from that album

In [4]:
%%sql
WITH new_table AS (SELECT 
        il.track_id, 
        tm.tname track_name, 
        aa.name artist_name, 
        tm.mname track_type, 
        il.unit_price, 
        il.quantity, 
        aa.title album 
    FROM invoice_line il 
    INNER JOIN (SELECT 
                    t.track_id, 
                    t.name tname, 
                    m.name mname 
                FROM track t 
                INNER JOIN media_type m ON t.media_type_id = m.media_type_id) tm 
    ON il.track_id = tm.track_id 
    INNER JOIN (SELECT 
                    at.name, 
                    al.title, 
                    t.track_id 
                    FROM artist at 
                    INNER JOIN album al ON at.artist_id = al.artist_id 
                    INNER JOIN track t ON al.album_id = t.album_id) aa 
    ON tm.track_id = aa.track_id)

SELECT 
    album, 
    artist_name artist, 
    COUNT(*) tracks_purchased 
FROM new_table
GROUP BY album 
ORDER BY tracks_purchased DESC 
LIMIT 5;

Done.


album,artist,tracks_purchased
Are You Experienced?,Jimi Hendrix,187
Faceless,Godsmack,96
Mezmerize,System Of A Down,93
Get Born,JET,90
The Doors,The Doors,83


### Selecting Albums to Purchase
The Chinook record store has just signed a deal with a new record label, and you've been tasked with selecting the **first three albums** that will be added to the store, **from a list of four**. All four albums are by artists that don't have any tracks in the store right now - we have the artist names, and the genre of music they produce:

    --------------------------------------
    |     Artist Name       |    Genre   |
    --------------------------------------
    |     Regal             |   Hip-Hop  |
    --------------------------------------
    |    Real Tone          |    Punk    |
    --------------------------------------
    | Meteror and the Girls |     Pop    |
    --------------------------------------
    |    Slim Jim Bites     |    Blues   |
    --------------------------------------
    
The record label specializes in artists from the **USA**, and they have given Chinook some money to advertise the new albums in the USA, so **we're interested in finding out which `genres` sell the best in the USA**.

1. Write a query that returns each **genre**, with the **number of tracks sold in the USA**:
    * in **absolute numbers**
    * in **percentages**.
2. Write a paragraph that interprets the data and makes a recommendation for the three artists whose albums we should purchase for the store, based on sales of tracks from their genres.

In [5]:
%%sql
WITH 
    genre_usa AS
            (SELECT 
                g.name genre,
                g.genre_id genre_id
            FROM genre g
            INNER JOIN track t ON t.genre_id = g.genre_id
            INNER JOIN invoice_line il ON t.track_id = il.track_id
            INNER JOIN invoice i ON il.invoice_id = i.invoice_id
            WHERE i.billing_country = 'USA'
            )
SELECT
        genre,
        COUNT(genre_id) number_of_tracks,
        ROUND(CAST(COUNT(genre_id) AS Float)/(SELECT COUNT(*) FROM genre_usa) * 100, 2) pct_of_track_sold
        FROM genre_usa
        GROUP BY genre
        ORDER by number_of_tracks DESC
        LIMIT 10;

Done.


genre,number_of_tracks,pct_of_track_sold
Rock,561,53.38
Alternative & Punk,130,12.37
Metal,124,11.8
R&B/Soul,53,5.04
Blues,36,3.43
Alternative,35,3.33
Latin,22,2.09
Pop,22,2.09
Hip Hop/Rap,20,1.9
Jazz,14,1.33


Among the **top 10 genres** are **Alternative & Punk (with `12.37%`market share)**, **Blues (with `3.43%`market share)**, **Pop (with `2.09%`market share)** and **Hip Hop/Rap (with `1.9%`market share)** in this order. We can therefore recommend the following **three artists** whose albums can be purchased for the store:

1. **Red Tone** (genre Punk)
2. **Slim Jim Bites** (genre Blues)
3. **Meteor and the Girls** (genre Pop)

## Analyzing Employee Sales Performance
**Each customer for the Chinook store gets assigned to a sales support agent** within the company when they first make a purchase. Analyze the purchases of customers belonging to each employee (sales support agent) to see if **any sales support agent is performing either better or worse than the others**.

1. Write a query that finds the **total dollar amount of sales assigned to each sales support agent** within the company. Add any extra attributes for that employee that you find are relevant to the analysis.
2. Write a short statement describing your results, and providing a possible interpretation.

In [6]:
%%sql
SELECT 
    e.first_name ||' '||e.last_name employee_name,
    ROUND(SUM(i.total), 2) total_dollar_generated,
    COUNT(c.customer_id) num_customers_per_agent 

FROM employee e 
INNER JOIN customer c ON e.employee_id = c.support_rep_id
INNER JOIN invoice i ON i.customer_id = c.customer_id 
GROUP BY employee_name
HAVING e.title = 'Sales Support Agent' 
ORDER BY total_dollar_generated DESC;

Done.


employee_name,total_dollar_generated,num_customers_per_agent
Jane Peacock,1731.51,212
Margaret Park,1584.0,214
Steve Johnson,1393.92,188


### Summary
* The Sales Agent **Jane Peacock** has genrated **`$1731.51`** from **`212`** customers. 
* The Sales Agent **Margaret Park** follows with **`$1584.0`** from **`214`** customers. 
* While Sales Agent **Steve Johnson** has genrated **`$1393.92`** from **`188`** customers.

**The analysis could be extended to calculated the percentage of income generated by each Sales agent. The time period taken to genrate the dollar amount could also be taken into consideration.**

### Alternatively
Without the pre-calculated **total** column in the **invoice table** this would be the approach to follow (***my opinion***).

In [7]:
%%sql
WITH 
    purchases AS (SELECT 
                 SUM(il.quantity * il.unit_price) total, 
                 c.customer_id,
                 c.support_rep_id
                 FROM invoice_line il 
                 INNER JOIN invoice i ON il.invoice_id = i.invoice_id
                 INNER JOIN customer c ON c.customer_id = i.customer_id
                 GROUP BY c.customer_id)
    
SELECT 
    e.first_name ||' '||e.last_name employee_name,
    ROUND(SUM(p.total), 2) total_dollar_generated,
    COUNT(p.customer_id) num_customers_per_agent 

FROM employee e 
INNER JOIN purchases p ON e.employee_id = p.support_rep_id 
GROUP BY employee_name
HAVING e.title = 'Sales Support Agent' 
ORDER BY total_dollar_generated DESC;

Done.


employee_name,total_dollar_generated,num_customers_per_agent
Jane Peacock,1731.51,21
Margaret Park,1584.0,20
Steve Johnson,1393.92,18


**The logic above needs some checking...especially the number of customers per sales agent**

## Analyzing Sales by Country
The next task is to **analyze the sales data for customers from each different country**. You have been given guidance to **use the country value from the customers table**, and **ignore the country from the billing address in the invoice table**. For each country, determine:

* total number of customers
* total value of sales
* average value of sales per customer
* average order value

In [8]:
%%sql
WITH 
    customers_by_country AS (SELECT 
                                 customer_id,
                                 country, 
                                 COUNT(customer_id) number_of_customers 
                             FROM customer
                             GROUP BY country
                             ORDER BY number_of_customers
                            ),

    purchase AS (SELECT 
                 ROUND(SUM(il.quantity * il.unit_price), 2) total_sales_value,
                 
                 c.customer_id
                 FROM invoice_line il 
                 INNER JOIN invoice i ON il.invoice_id = i.invoice_id
                 INNER JOIN customer c ON c.customer_id = i.customer_id
                 GROUP BY c.customer_id)
                 
SELECT 
    c_by_c.country,
    c_by_c.number_of_customers,
    p.total_sales_value,
    ROUND(p.total_sales_value / c_by_c.number_of_customers, 2) avg_sales_per_customer
    FROM customers_by_country c_by_c
    INNER JOIN purchase p ON p.customer_id = c_by_c.customer_id
    GROUP BY c_by_c.country
    ORDER BY number_of_customers DESC
    LIMIT 10;

Done.


country,number_of_customers,total_sales_value,avg_sales_per_customer
USA,13,72.27,5.56
Canada,8,75.24,9.4
Brazil,5,106.92,21.38
France,5,73.26,14.65
Germany,4,73.26,18.32
United Kingdom,3,79.2,26.4
Czech Republic,2,128.7,64.35
India,2,71.28,35.64
Portugal,2,82.17,41.09
Argentina,1,39.6,39.6


## Albums vs Individual Tracks
The Chinook store is setup in a way that allows customer to make purchases in one of the two ways:

1. purchase a whole album
2. purchase a collection of one or more individual tracks.


* The store does not allow customers purchase a whole album, and then add individual tracks to that same purchase (but customers can purchase all tracks in an album by choosing each track manually. This case is rare and should be ignored in the analysis).


* Albums with very few tracks should also be ignored in the analysis since most customers might tend to buy such whole albums.


* When customers purchase albums they are charged the same price as if they had purchased each of those tracks separately.


* To save money, companies are considering to purchase only the most popular tracks from each album instead of purchasing every track from an album.


**We have been asked to find out `what percentage of purchases are individual tracks vs whole albums`, so that management can use this data to understand the effect this decision might have on overall revenue.**


* In order to answer the question, we're going to have to **identify whether each invoice has all the tracks from an album**. We can do this by getting the list of tracks from an invoice and comparing it to the list of tracks from an album.


* We can find an album by looking up the album that one of the purchased tracks belongs to.

### The Approach
1. Write a query that categorizes each invoice as either an album purchase or not, and calculates the following summary statistics:

    * Number of invoices
    * Percentage of invoices 
    
2. Write one to two sentences explaining your findings, and making a prospective recommendation on whether the Chinook store should continue to buy full albums from record companies

In [16]:
%%sql
DROP VIEW IF EXISTS tracks_by_invoice;

CREATE VIEW tracks_by_invoice AS
    SELECT 
        il.invoice_id,
        COUNT(t.track_id) Tracks_on_Invoice,
        t.album_id album_id
    FROM invoice_line il
    INNER JOIN track t on t.track_id = il.track_id
    GROUP BY 1,3 ORDER BY 1;

SELECT
    tracks_by_invoice.*,
    tracks_by_album.Tracks_on_Album
    FROM tracks_by_invoice
    INNER JOIN tracks_by_album on tracks_by_album.album_id = tracks_by_invoice.album_id;

DROP VIEW IF EXISTS album_purchase_count;

CREATE VIEW album_purchase_count AS
    SELECT 
        tracks_by_invoice.invoice_id, 
        CASE    
            WHEN(
                tracks_by_invoice.Tracks_on_Invoice = tracks_by_album.Tracks_on_Album
                AND NOT EXISTS (
                    SELECT 1 FROM invoice_line il
                    LEFT JOIN track t ON t.track_id = il.track_id
                    WHERE il.invoice_id = tracks_by_invoice.invoice_id
                    AND t.album_id = tracks_by_album.album_id
                    AND t.track_id NOT IN (
                        SELECT track_id FROM invoice_line
                        WHERE invoice_id = tracks_by_invoice.invoice_id
                    )
                )
            ) THEN 'Album Purchase'       
            ELSE 'Individual Tracks'    
        END AS album_purchase_or_not
    FROM tracks_by_invoice
    INNER JOIN tracks_by_album ON tracks_by_album.album_id = tracks_by_invoice.album_id;
    

SELECT 
    COUNT(invoice_id) number_of_invoices,
    ROUND(CAST(COUNT(invoice_id) AS FLOAT)/ (SELECT COUNT(invoice_id)
                       FROM invoice) * 100, 2) pct_of_invoices
    FROM album_purchase_count
    WHERE album_purchase_or_not = 'Album Purchase';

Done.
Done.
Done.
Done.
Done.
Done.


number_of_invoices,pct_of_invoices
30,4.89
